In [1]:
import os
os.environ["SPARK_HOME"] = "/Applications/spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_PYTHON"] = "python"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-ml") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

26/01/24 23:04:14 WARN Utils: Your hostname, Kavis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.24 instead (on interface en0)
26/01/24 23:04:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/kavisanthoshkumar/.ivy2/cache
The jars for the packages stored in: /Users/kavisanthoshkumar/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a5e1471-fd02-436e-9b73-4d731fc6c9b3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central


:: loading settings :: url = jar:file:/Applications/Spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 111ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-4a5e1471-fd02-436e-9b73-4d731fc6c9b3
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/2ms)
26/01/24 23:04:15 WARN NativeCodeLoader: Unable to load native-

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, 
    StructField, 
    IntegerType, 
    FloatType, 
    StringType, 
    DoubleType, 
    BooleanType
)
from pyspark.sql import DataFrame

#### 1.	Load the CSV file with header and infer schema.

In [4]:
data = spark.read.csv("../dataset/online_shoppers_intention.csv", header = True, inferSchema = True)

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



#### 2.	Load the CSV again using an explicitly defined schema.

In [5]:
data_schema = StructType([
    StructField("Administrative", IntegerType()), 
    StructField("Administrative_Duration", DoubleType()), 
    StructField("Informational", IntegerType()), 
    StructField("Informational_Duration", DoubleType()), 
    StructField("ProductRelated", IntegerType()), 
    StructField("ProductRelated_Duration", DoubleType()), 
    StructField("BounceRates", DoubleType()), 
    StructField("ExitRates", DoubleType()), 
    StructField("PageValues", DoubleType()), 
    StructField("SpecialDay", DoubleType()),
    StructField("Month", StringType()), 
    StructField("OperatingSystems", IntegerType()), 
    StructField("Browser", IntegerType()),
    StructField("Region", IntegerType()), 
    StructField("TrafficType",IntegerType()), 
    StructField("VisitorType", StringType()), 
    StructField("Weekend", BooleanType()),
    StructField("Revenue", BooleanType())
])


data = spark.read.csv("../dataset/online_shoppers_intention.csv", schema = data_schema, header = True)

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



#### 3.	Add a column ingestion_timestamp using current timestamp.

In [6]:
current_date = datetime.now().date()
print(f"Ingestion Date : {current_date}")

Ingestion Date : 2026-01-24


In [7]:
data = data.withColumn("ingestion_timestamp", F.lit(current_date))

data.select('ingestion_timestamp').show()

+-------------------+
|ingestion_timestamp|
+-------------------+
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
|         2026-01-24|
+-------------------+
only showing top 20 rows



#### 4.	Rename column Revenue to made_purchase.

In [11]:
data = data.withColumnRenamed("Revenue", "made_purchase")
data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- made_purchase: boolean (nullable = true)
 |-- ingestion_timestamp: date (nullable = false)



#### 5. Convert Month to lowercase

In [ ]:
data = data.withColumn("Month", F.lower(F.col("Month")))

data.select("Month").show() # all the entries are converted to lowercase characters

+-----+
|Month|
+-----+
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
|  feb|
+-----+
only showing top 20 rows



#### 6.	Drop records where VisitorType is null.

In [ ]:
data.select("VisitorType").distinct().show()
# We can see only three different categories in VisitorType column 

+-----------------+
|      VisitorType|
+-----------------+
|      New_Visitor|
|            Other|
|Returning_Visitor|
+-----------------+



In [ ]:
# Dropping th records where VisitorType is null
data = data[F.col("VisitorType").isNull()]
# Currently we don't have any records with null entires     

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|made_purchase|ingestion_timestamp|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-------

#### 7.	Count total number of sessions.

In [24]:
data.select("sessions").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `sessions` cannot be resolved. Did you mean one of the following? [`Region`, `Month`, `Weekend`, `Browser`, `ExitRates`].;
'Project ['sessions]
+- Project [Administrative#53, Administrative_Duration#54, Informational#55, Informational_Duration#56, ProductRelated#57, ProductRelated_Duration#58, BounceRates#59, ExitRates#60, PageValues#61, SpecialDay#62, Month#63, OperatingSystems#64, Browser#65, Region#66, TrafficType#67, VisitorType#68, Weekend#69, Revenue#70 AS made_purchase#238, ingestion_timestamp#89]
   +- Project [Administrative#53, Administrative_Duration#54, Informational#55, Informational_Duration#56, ProductRelated#57, ProductRelated_Duration#58, BounceRates#59, ExitRates#60, PageValues#61, SpecialDay#62, Month#63, OperatingSystems#64, Browser#65, Region#66, TrafficType#67, VisitorType#68, Weekend#69, Revenue#70, 2026-01-24 AS ingestion_timestamp#89]
      +- Relation [Administrativ

In [25]:
data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- made_purchase: boolean (nullable = true)
 |-- ingestion_timestamp: date (nullable = false)

